## Evaluating regression techniques for speaker characterization
### Laura Fernández Gallardo

Multioutput regression of the 7 items contributing to the [perception](https://github.com/laufergall/Subjective_Speaker_Characteristics) of the 'warmth' trait:

warmth (males) = 
0.85 x hearty + 0.84 x affectionate - 0.76 x distant + 0.59 x friendly - 0.58 x unsympathetic - 0.52 x non_likable + 0.51 x not_irritated 

warmth (females) = 
0.84 x hearty + 0.84 x affectionate - 0.78 x distant + 0.56 x friendly - 0.49 x unsympathetic + 0.49 x not_irritated - 0.45 x non_likable 

In [1]:
import io
import requests

import numpy as np
import pandas as pd

from reg_tuning import * # my helper functions

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# fix random seed for reproducibility
seed = 2302
np.random.seed(seed)

## Load features and labels

In [3]:
path = 'https://raw.githubusercontent.com/laufergall/ML_Speaker_Characteristics/master/data/generated_data/'

url = path + "feats_ratings_scores_train.csv"
s = requests.get(url).content
feats_ratings_scores_train = pd.read_csv(io.StringIO(s.decode('utf-8')))

url = path + "feats_ratings_scores_test.csv"
s = requests.get(url).content
feats_ratings_scores_test = pd.read_csv(io.StringIO(s.decode('utf-8')))

with open(r'..\data\generated_data\feats_names.txt') as f:
    feats_names = f.readlines()
feats_names = [x.strip().strip('\'') for x in feats_names] 

with open(r'..\data\generated_data\items_names.txt') as f:
    items_names = f.readlines()
items_names = [x.strip().strip('\'') for x in items_names] 

with open(r'..\data\generated_data\traits_names.txt') as f:
    traits_names = f.readlines()
traits_names = [x.strip().strip('\'') for x in traits_names] 

## 7warmthitems: Model tuning

Use the train data to find the regressor and its hyperparameters leading to the best performance. 

Not performing feature selection with "SelectKBest": Univariate feature selection does not support multilabel. 

In [4]:
# Standardize speech features  

dropcolumns = ['name','spkID','speaker_gender'] + items_names + traits_names

# learn transformation on training data
scaler = StandardScaler()
scaler.fit(feats_ratings_scores_train.drop(dropcolumns, axis=1))

# numpy n_instances x n_feats
feats_s_train = scaler.transform(feats_ratings_scores_train.drop(dropcolumns, axis=1))
feats_s_test = scaler.transform(feats_ratings_scores_test.drop(dropcolumns, axis=1)) 

In [5]:
warmthitems_names = ['hearty','affectionate','distant','friendly','unsympathetic','non_likable','not_irritated']

In [6]:
# training data. Features and labels
X = feats_s_train # (2700, 88)
y = feats_ratings_scores_train[warmthitems_names].as_matrix() # (2700, 7)

# test data. Features and labels
Xt = feats_s_test # (891, 88)
yt = feats_ratings_scores_test[warmthitems_names].as_matrix() # (891, 7)

# split train data into 80% and 20% subsets - with balance in trait and gender
# give subset A to the inner hyperparameter tuner
# and hold out subset B for meta-evaluation
AX, BX, Ay, By = train_test_split(X, y, 
                                  test_size=0.20, 
                                  stratify = feats_ratings_scores_train['speaker_gender'], 
                                  random_state=2302)

In [7]:
# dataframe with results from hp tuner to be appended
tuning_all = pd.DataFrame()

# list with tuned models trained on training data, to be appended
trained_all = []

In [8]:
# save splits

label = 'multioutput_7warmthitems'

# train/test partitions, features and labels
np.save(r'.\data_while_tuning\X_' + label + '.npy', X)
np.save(r'.\data_while_tuning\y_' + label + '.npy', y)
np.save(r'.\data_while_tuning\Xt_' + label + '.npy', Xt)
np.save(r'.\data_while_tuning\yt_' + label + '.npy', yt)

# # A/B splits, features and labels
np.save(r'.\data_while_tuning\AX_' + label + '.npy', AX)
np.save(r'.\data_while_tuning\BX_' + label + '.npy', BX)
np.save(r'.\data_while_tuning\Ay_' + label + '.npy', Ay)
np.save(r'.\data_while_tuning\By_' + label + '.npy', By)

### Calling hp_tuner() for each regressor

** Recover ** when new ipynb session started.

(Workaround for working with hyperparameter tuning during several days)

In [9]:
# label = 'multioutput_7warmthitems'

# # train/test partitions, features and labels
# X = np.load(r'.\data_while_tuning\X_' + label + '.npy')
# y = np.load(r'.\data_while_tuning\y_' + label + '.npy')
# Xt = np.load(r'.\data_while_tuning\Xt_' + label + '.npy')
# yt = np.load(r'.\data_while_tuning\yt_' + label + '.npy')

# # A/B splits, features and labels
# AX = np.load(r'.\data_while_tuning\AX_' + label + '.npy')
# BX = np.load(r'.\data_while_tuning\BX_' + label + '.npy')
# Ay = np.load(r'.\data_while_tuning\Ay_' + label + '.npy')
# By = np.load(r'.\data_while_tuning\By_' + label + '.npy')


In [10]:
# # Loading outpus of hp tuning from disk
# tuning_all, trained_all = load_tuning(label)

Call this after each experiment **to recover later**: 

In [11]:
# # save tuning_all (.csv) and trained_all (namemodel.sav)
# save_tuning(tuning_all, trained_all, label)

#### Random Forest

*class sklearn.ensemble.RandomForestRegressor(n_estimators=10, criterion=’mse’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)*

Tune: max_features, max_depth, min_samples_leaf

In [12]:
from sklearn.ensemble import RandomForestRegressor

"""
Random Forest
"""
def get_RandomForestRegressor2tune():
    
    model = RandomForestRegressor(random_state=2302, max_features = None)
    hp = dict(
        regressor__estimator__max_features = np.arange(2,50),
        regressor__estimator__max_depth = np.arange(2,50), 
        regressor__estimator__min_samples_leaf = np.arange(2,50) 
    )
    return 'RandomForestRegressor', model, hp

# tune this model (multiputput)
tuning, trained = hp_tuner(AX, BX, Ay, By, 
                           [get_RandomForestRegressor2tune], 
                           label,
                           feats_names,
                           [88], # feature selection not performed
                           mode='random',
                           n_iter=50
                          )

'RandomForestRegressor' -> Best cross-val score on A set: -55.505871 using {'regressor__estimator__min_samples_leaf': 2, 'regressor__estimator__max_features': 40, 'regressor__estimator__max_depth': 22}
'RandomForestRegressor' -> root mean_squared_error on B set: 7.221159



In [13]:
from sklearn.ensemble import RandomForestRegressor

"""
Random Forest
"""
def get_RandomForestRegressor2tune():
    
    model = RandomForestRegressor(random_state=2302, max_features = None)
    hp = dict(
        regressor__estimator__max_features = np.arange(30,50),
        regressor__estimator__max_depth = np.arange(15,35), 
        regressor__estimator__min_samples_leaf = np.arange(2,5) 
    )
    return 'RandomForestRegressor', model, hp

# tune this model (multiputput)
tuning, trained = hp_tuner(AX, BX, Ay, By, 
                           [get_RandomForestRegressor2tune], 
                           label,
                           feats_names,
                           [88], # feature selection not performed
                           mode='random',
                           n_iter=50
                          )

'RandomForestRegressor' -> Best cross-val score on A set: -55.330047 using {'regressor__estimator__min_samples_leaf': 2, 'regressor__estimator__max_features': 32, 'regressor__estimator__max_depth': 26}
'RandomForestRegressor' -> root mean_squared_error on B set: 7.132426



In [14]:
# update lists of tuning info and trained regressors
tuning_all = tuning_all.append(tuning, ignore_index=True)
trained_all.append(trained)

# save tuning_all (.csv) and trained_all (nameregressor.sav)
save_tuning(tuning_all, trained_all, label)

#### Decision tree

*class sklearn.tree.DecisionTreeRegressor(criterion=’mse’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, presort=False)*

Tune: max_depth

In [15]:
from sklearn.tree import DecisionTreeRegressor

"""
Decision Trees
"""
def get_DecisionTreeRegressor2tune():
    
    model = DecisionTreeRegressor()
    hp = dict(
        regressor__estimator__max_depth = np.arange(2,30), 
        regressor__estimator__max_features = np.arange(2,50)
    )
    return 'DecisionTreeRegressor', model, hp

# tune this model (multiputput)
tuning, trained = hp_tuner(AX, BX, Ay, By, 
                           [get_DecisionTreeRegressor2tune], 
                           label,
                           feats_names,
                           [88], # feature selection not performed
                           mode='random',
                           n_iter=30
                          )

'DecisionTreeRegressor' -> Best cross-val score on A set: -78.252017 using {'regressor__estimator__max_features': 30, 'regressor__estimator__max_depth': 4}
'DecisionTreeRegressor' -> root mean_squared_error on B set: 8.468760



In [16]:
from sklearn.tree import DecisionTreeRegressor

"""
Decision Trees
"""
def get_DecisionTreeRegressor2tune():
    
    model = DecisionTreeRegressor()
    hp = dict(
        regressor__estimator__max_depth = np.arange(2,6), 
        regressor__estimator__max_features = np.arange(20,40)
    )
    return 'DecisionTreeRegressor', model, hp

# tune this model (multiputput)
tuning, trained = hp_tuner(AX, BX, Ay, By, 
                           [get_DecisionTreeRegressor2tune], 
                           label,
                           feats_names,
                           [88], # feature selection not performed
                           mode='random',
                           n_iter=30
                          )

'DecisionTreeRegressor' -> Best cross-val score on A set: -76.493110 using {'regressor__estimator__max_features': 39, 'regressor__estimator__max_depth': 4}
'DecisionTreeRegressor' -> root mean_squared_error on B set: 8.460729



In [17]:
# update lists of tuning info and trained regressors
tuning_all = tuning_all.append(tuning, ignore_index=True)
trained_all.append(trained)

# save tuning_all (.csv) and trained_all (nameregressor.sav)
save_tuning(tuning_all, trained_all, label)

Add baseline performance.

In [18]:
from sklearn.dummy import DummyRegressor

model = DummyRegressor(strategy='mean')
model.fit(AX, Ay)
By_pred = model.predict(BX)
score_on_B = np.sqrt(mean_squared_error(By, By_pred))
d = {
    'regressors_names': ['DummyRegressor'],
    'best_accs': score_on_B,
    'best_hps': '',
    'sel_feats': '',
    'sel_feats_i': ''
    }

tuning = pd.DataFrame(data = d)
trained = model.fit(X, y)

# update lists of tuning info and trained regressors
tuning_all = tuning_all.append(tuning, ignore_index=True)
trained_all.append([trained])

# save tuning_all (.csv) and trained_all (nameregressor.sav)
save_tuning(tuning_all, trained_all, label)

## Testing

### RMSE
Testing the best model found after hyperparameter tuning.

Performance metric: RMSE.
    
The predictions correspond to the scores of each test segment (3 parts x 4 dialogs) spoken by the same test speaker. I perform the average of the predicted scores that correspond to the same speaker - to be compared to the true scores.

Visualization: pairplot of scores of true and predicted test data, grouped-averaged by speakers.

Load data if new ipynb session:

In [3]:
label = 'multioutput_7warmthitems'


path = 'https://raw.githubusercontent.com/laufergall/ML_Speaker_Characteristics/master/data/generated_data/'

url = path + "feats_ratings_scores_train.csv"
s = requests.get(url).content
feats_ratings_scores_train = pd.read_csv(io.StringIO(s.decode('utf-8')))

url = path + "feats_ratings_scores_test.csv"
s = requests.get(url).content
feats_ratings_scores_test = pd.read_csv(io.StringIO(s.decode('utf-8')))

with open(r'..\data\generated_data\feats_names.txt') as f:
    feats_names = f.readlines()
feats_names = [x.strip().strip('\'') for x in feats_names] 

with open(r'..\data\generated_data\items_names.txt') as f:
    items_names = f.readlines()
items_names = [x.strip().strip('\'') for x in items_names] 

with open(r'..\data\generated_data\traits_names.txt') as f:
    traits_names = f.readlines()
traits_names = [x.strip().strip('\'') for x in traits_names] 

# train/test partitions, features and labels
X = np.load(r'.\data_while_tuning\X_' + label + '.npy')
y = np.load(r'.\data_while_tuning\y_' + label + '.npy')
Xt = np.load(r'.\data_while_tuning\Xt_' + label + '.npy')
yt = np.load(r'.\data_while_tuning\yt_' + label + '.npy')

# Loading outpus of hp tuning from disk
tuning_all, trained_all = load_tuning(label)
tuning_all

,best_accs,best_hps,regressors_names,sel_feats,sel_feats_i
0,7.132426,"{'regressor__estimator__min_samples_leaf': 2, ...",RandomForestRegressor,"['F0semitoneFrom27.5Hz_sma3nz_amean', 'F0semit...",[ True True True True True True True Tr...
1,8.460729,"{'regressor__estimator__max_features': 39, 're...",DecisionTreeRegressor,"['F0semitoneFrom27.5Hz_sma3nz_amean', 'F0semit...",[ True True True True True True True Tr...
2,8.933428,NaN,DummyRegressor,NaN,NaN


In [6]:
warmthitems_names = ['hearty','affectionate','distant','friendly','unsympathetic','non_likable','not_irritated']

In [7]:
# select the classifier that gave the maximum acc on B set
best_accs = tuning_all['best_accs']
i_best = best_accs.idxmin()

print('Selected classifier based on the best performance on B: %r (perf. on B = %0.2f)' % (tuning_all.loc[i_best,'regressors_names'], round(best_accs[i_best],2)))

Selected classifier based on the best performance on B: 'RandomForestRegressor' (perf. on B = 7.13)


In [8]:
from sklearn.metrics import r2_score

# go through performace for all regressors

# removing duplicates from tuning_all (same classifier tuned twice with different searchers)
indexes = tuning_all['regressors_names'].drop_duplicates(keep='last').index.values

# dataframe for summary of performances
# performances = pd.DataFrame(tuning_all.loc[indexes,['regressors_names','best_accs']])

for i in indexes:

    # compute predictions with the best tuned regressor

    yt_pred = trained_all[i][0].predict(Xt)

    # average of outputs that belong to the same speaker

    true_scores = pd.DataFrame(data = feats_ratings_scores_test[warmthitems_names+['spkID']])
    true_scores['type']='true'

    pred_scores=pd.DataFrame()
    for t in np.arange(0,len(warmthitems_names)):
        pred_scores[warmthitems_names[t]] = yt_pred[:, t] 
    pred_scores['spkID'] = feats_ratings_scores_test['spkID']


    # group by speakers and average
    true_scores_avg = true_scores.groupby('spkID').mean()

    pred_scores_avg = pred_scores.groupby('spkID').mean()


    # RMSE and R2 for each trait separately
    for t in np.arange(0,len(warmthitems_names)):
        print('%r -> avg RMSE %r = %.2f' % (tuning_all.loc[i,'regressors_names'],
                                      warmthitems_names[t], 
                                      np.sqrt(mean_squared_error(true_scores_avg[warmthitems_names[t]].as_matrix(), 
                                                                                  pred_scores_avg[warmthitems_names[t]].as_matrix()))
                                     )
             )
        
        print('%r -> avg R2 %r = %.2f' % (tuning_all.loc[i,'regressors_names'],
                                      warmthitems_names[t], 
                                      r2_score(true_scores_avg[warmthitems_names[t]].as_matrix(), 
                                                                                  pred_scores_avg[warmthitems_names[t]].as_matrix())
                                     )
             )

    # overall RMSE and R2
    myrmse_avg = np.sqrt(mean_squared_error(true_scores_avg[warmthitems_names].as_matrix(), 
                                            pred_scores_avg[warmthitems_names].as_matrix())
                        )
    myr2_avg = r2_score(true_scores_avg[warmthitems_names].as_matrix(), 
                                            pred_scores_avg[warmthitems_names].as_matrix()
                        )
    print('%r -> avg R2 overall: %0.2f' % (tuning_all.loc[i,'regressors_names'], myr2_avg))
    print('')
    
        
    # append true and predicted scores

    true_scores_avg.reset_index(inplace=True)
    pred_scores_avg.reset_index(inplace=True) 

    true_scores_avg['type']='true'
    pred_scores_avg['type']='pred'

    test_scores_avg=true_scores_avg.append(pred_scores_avg)

#     # pairplot color-coded by true/predicted test data
#     myfig = sns.pairplot(test_scores_avg.drop('spkID', axis=1), hue='type')

#     # save figure
#     filename = label + '_test_'+tuning_all.loc[i,'regressors_names']+'.png'
#     myfig.savefig('.\\figures\\' + filename, bbox_inches = 'tight')  


'RandomForestRegressor' -> avg RMSE 'hearty' = 9.67
'RandomForestRegressor' -> avg R2 'hearty' = 0.20
'RandomForestRegressor' -> avg RMSE 'affectionate' = 9.23
'RandomForestRegressor' -> avg R2 'affectionate' = 0.23
'RandomForestRegressor' -> avg RMSE 'distant' = 9.25
'RandomForestRegressor' -> avg R2 'distant' = 0.15
'RandomForestRegressor' -> avg RMSE 'friendly' = 8.57
'RandomForestRegressor' -> avg R2 'friendly' = 0.22
'RandomForestRegressor' -> avg RMSE 'unsympathetic' = 7.32
'RandomForestRegressor' -> avg R2 'unsympathetic' = 0.11
'RandomForestRegressor' -> avg RMSE 'non_likable' = 10.17
'RandomForestRegressor' -> avg R2 'non_likable' = 0.09
'RandomForestRegressor' -> avg RMSE 'not_irritated' = 9.57
'RandomForestRegressor' -> avg R2 'not_irritated' = 0.06
'RandomForestRegressor' -> avg R2 overall: 0.15

'DecisionTreeRegressor' -> avg RMSE 'hearty' = 9.67
'DecisionTreeRegressor' -> avg R2 'hearty' = 0.20
'DecisionTreeRegressor' -> avg RMSE 'affectionate' = 9.65
'DecisionTreeRegress

These are more or less the same results as when doing multioutput classification with the 34 items.